# 1.五點特徵人臉影像辨識

In [1]:
import dlib
import cv2
predictor = "shape_predictor_5_face_landmarks.dat"  #模型(5點)
sp = dlib.shape_predictor(predictor)  #讀入模型
detector = dlib.get_frontal_face_detector()  #偵測臉部

img = dlib.load_rgb_image("media\\LINE_230625.jpg")  #讀取圖片
win = dlib.image_window()  #建立顯示視窗
win.clear_overlay()  #清除圖形
win.set_image(img)  #顯示圖片

dets = detector(img, 1)  #臉部偵測,1為彩色
print("人臉數：{}".format(len(dets)))
#繪製人臉矩形及5點特徵
for k, det in enumerate(dets):
    print("偵測人臉 {}: 左：{}  上：{}  右：{}  下：{}".format(k, det.left(), det.top(), det.right(), det.bottom()))  #人臉坐標
    win.add_overlay(det)  #顯示矩形
    shape = sp(img, det)  #取得5點特徵 
    win.add_overlay(shape)  #顯示5點特徵
    dlib.hit_enter_to_continue()  #保持影像

人臉數：8
偵測人臉 0: 左：619  上：350  右：681  下：412
偵測人臉 1: 左：751  上：343  右：803  下：395
偵測人臉 2: 左：923  上：357  右：985  下：419
偵測人臉 3: 左：855  上：435  右：907  下：487
偵測人臉 4: 左：959  上：464  右：1011  下：516
偵測人臉 5: 左：654  上：475  右：705  下：527
偵測人臉 6: 左：536  上：357  右：598  下：419
偵測人臉 7: 左：377  上：370  右：439  下：433


# 2.CNN訓練模型人臉影像辨識

In [2]:
import dlib
import cv2

cnn_detector = dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")  #CNN模型
img = cv2.imread("media\\LINE_230625.jpg")
dets = cnn_detector(img, 1)  #偵測人臉
print("人臉數：{}".format(len(dets)))
for i, det in enumerate(dets):
    #det.rect是人臉矩形坐標,det.confidence為信心指數
    face = det.rect
    left = face.left()
    top = face.top()
    right = face.right()
    bottom = face.bottom()
    print("偵測人臉 {}: 左：{}  上：{}  右：{}  下：{}  信心指數：{}".format(i, left, top, right, bottom, det.confidence))
    cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 1)  #畫人臉矩形

cv2.namedWindow("win", cv2.WINDOW_AUTOSIZE)
cv2.imshow("win", img)
k = cv2.waitKey(0)
cv2.destroyAllWindows()

人臉數：8
偵測人臉 0: 左：923  上：358  右：980  下：415  信心指數：1.0898116827011108
偵測人臉 1: 左：854  上：433  右：911  下：490  信心指數：1.0861622095108032
偵測人臉 2: 左：654  上：472  右：701  下：520  信心指數：1.084273099899292
偵測人臉 3: 左：755  上：343  右：802  下：390  信心指數：1.0825084447860718
偵測人臉 4: 左：611  上：344  右：679  下：412  信心指數：1.0805684328079224
偵測人臉 5: 左：958  上：456  右：1015  下：513  信心指數：1.0772682428359985
偵測人臉 6: 左：543  上：362  右：591  下：409  信心指數：1.0736702680587769
偵測人臉 7: 左：369  上：358  右：437  下：426  信心指數：1.0615956783294678


# 3. 68點特徵人臉影像辨識

In [3]:
import numpy as np
import cv2  #影象處理庫OpenCV
import dlib

predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')   #構建68點特徵
detector = dlib.get_frontal_face_detector()  #偵測臉部正面

img = cv2.imread("media\\LINE_ALBUM_1120609.jpg")  #讀取影象
dets = detector(img, 1)  #偵測人臉
for det in dets:
    #人臉關鍵點識別
    landmarks = []
    for p in predictor(img, det).parts():  
        landmarks.append(np.matrix([p.x, p.y])) 
    # 取得68點座標
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])  #[0,0]為x坐標,[0,1]為y坐標
        cv2.circle(img, pos, 5, color=(0, 255, 0))  #畫出68個小圓點
        font = cv2.FONT_HERSHEY_SIMPLEX  # 利用cv2.putText輸出1-68
        #引數依次是：圖片，新增的文字，座標，字型，字型大小，顏色，字型粗細
        cv2.putText(img, str(idx+1), pos, font, 0.4, (0, 0, 255), 1,cv2.LINE_AA)

cv2.namedWindow("img", 2)     
cv2.imshow("img", img)       #顯示影象
cv2.waitKey(0)
cv2.destroyWindow("img")

# 4.攝影機圖像中劃出點輪廓

In [9]:
from imutils.video import VideoStream
from imutils import face_utils
import imutils
import time
import dlib
import cv2

detector=dlib.get_frontal_face_detector()  #偵測臉部正面
predictor=dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  #構建68點特徵

#左右眼特徵點索引
(left_Start,left_End)=face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(right_Start,right_End)=face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
#嘴特徵點索引
(leftMouth,rightMouth)=face_utils.FACIAL_LANDMARKS_IDXS['mouth']
#下巴特徵點索引
(leftJaw,rightJaw)=face_utils.FACIAL_LANDMARKS_IDXS['jaw']
#鼻子特徵點索引
(leftNose,rightNose)=face_utils.FACIAL_LANDMARKS_IDXS['nose']
#左右眉毛特徵點索引
(left_leftEyebrow,left_rightEyebrow)=face_utils.FACIAL_LANDMARKS_IDXS['left_eyebrow']
(right_leftEyebrow,right_rightEyebrow)=face_utils.FACIAL_LANDMARKS_IDXS['right_eyebrow']

vsThread=VideoStream(src=0).start()  #開啟攝影機
time.sleep(2.0)

while True:
    frame = vsThread.read()  #讀取影格
    frame = imutils.resize(frame, width=720)
    faces = detector(frame, 0)  #偵測人臉
    for face in faces:
        shape = predictor(frame, face)  #取得人臉
        shape = face_utils.shape_to_np(shape)  #轉為numpy
        #左右眼特徵點
        leftEye = shape[left_Start:left_End]
        rightEye = shape[right_Start:right_End]
        #轉為外殼
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        #畫出輪廓
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        #嘴
        mouth=shape[leftMouth:rightMouth]
        mouthHull=cv2.convexHull(mouth)
        cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
        #鼻子
        nose=shape[leftNose:rightNose]
        noseHull=cv2.convexHull(nose)
        cv2.drawContours(frame, [noseHull], -1, (0, 255, 0), 1)
        #下巴
        jaw=shape[leftJaw:rightJaw]
        jawHull=cv2.convexHull(jaw)
        cv2.drawContours(frame, [jawHull], -1, (0, 255, 0), 1)
        #眉毛
        leftEyebrow=shape[left_leftEyebrow:left_rightEyebrow]
        rightEyebrow=shape[right_leftEyebrow:right_rightEyebrow]
        leftEyebrowHull=cv2.convexHull(leftEyebrow)
        rightEyebrowHull=cv2.convexHull(rightEyebrow)
        cv2.drawContours(frame, [leftEyebrowHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyebrowHull], -1, (0, 255, 0), 1)

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# 5.攝影機臉部偵測

In [11]:
import dlib
import cv2
import imutils
# 開啟影片檔案
cap = cv2.VideoCapture(0)
# Dlib 的人臉偵測器
detector = dlib.get_frontal_face_detector()
# 以迴圈從影片檔案讀取影格，並顯示出來
while(cap.isOpened()):
  ret, frame = cap.read()
  # 偵測人臉
  face_rects, scores, idx = detector.run(frame, 0)
  # 取出所有偵測的結果
  for i, d in enumerate(face_rects):
    x1 = d.left()
    y1 = d.top()
    x2 = d.right()
    y2 = d.bottom()
    text = "%2.2f(%d)" % (scores[i], idx[i])
    # 以方框標示偵測的人臉
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4, cv2.LINE_AA)
    # 標示分數
    cv2.putText(frame, text, (x1, y1), cv2.FONT_HERSHEY_DUPLEX,
            0.7, (255, 255, 255), 1, cv2.LINE_AA)
  # 顯示結果
  cv2.imshow("Face Detection", frame)
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break
cap.release()
cv2.destroyAllWindows()

# 6.辨識不同的臉

In [12]:
import dlib, numpy

predictor = "shape_predictor_68_face_landmarks.dat"  #人臉68特徵點模型
recogmodel = "dlib_face_recognition_resnet_model_v1.dat"  #人臉辨識模型

detector = dlib.get_frontal_face_detector()  #偵測臉部正面
sp = dlib.shape_predictor(predictor)  #讀入人臉特徵點模型
facerec = dlib.face_recognition_model_v1(recogmodel)  #讀入人臉辨識模型

#取得人臉特徵點向量
def getFeature(imgfile):
    img = dlib.load_rgb_image(imgfile)
    dets = detector(img, 1)
    for det in dets:
        shape = sp(img, det)  #特徵點偵測
        feature = facerec.compute_face_descriptor(img, shape)  #取得128維特徵向量
        return numpy.array(feature)  #轉換numpy array格式

#判斷是否同一人 
def samePerson(pic1, pic2):
    feature1 = getFeature(pic1)
    feature2 = getFeature(pic2)
    dist = numpy.linalg.norm(feature1-feature2)  # 計算歐式距離,越小越像
    print("歐式距離={}".format(dist))
    if dist < 0.3: 
        print("{} 和 {} 為同一個人！".format(pic1, pic2))
    else:
        print("{} 和 {} 不是同一個人！".format(pic1, pic2))
    print()
    
samePerson("media\\LINE_ALBUM_111.jpg", "media\\LINE_ALBUM_11106.jpg")  #不同人
samePerson("media\\LINE_ALBUM_1125.jpg", "media\\LINE_ALBUM_1120610.jpg")  #同一人


歐式距離=0.2952158570099693
media\LINE_ALBUM_111.jpg 和 media\LINE_ALBUM_11106.jpg 為同一個人！

歐式距離=0.38403234459641017
media\LINE_ALBUM_1125.jpg 和 media\LINE_ALBUM_1120610.jpg 不是同一個人！

